# Sidekick - Output as input with non-normalized data

#### Model
We change completely the approach. Instead of using the time as input and trying to predict the output at new time indices, we now consider the pledged money at each time step as input ($\mathbf{y}$ becomes $\mathbf{x}$) and the last time index as the output. That is, we have now a dataset $\mathcal{D} = \left\{ (\mathbf{x}^{(p)}, y^{(p)}) \mid p = 1, ..., P \right\}$ with $\mathbf{x}^{(p)} = \mathbf{y}_{1:t}^{(p)}$ and $\mathbf{y}^{(p)} = y_T^{(p)}$. We then have $X = \left[\mathbf{x}^{(p)}\right]_{p=1}^P$ a $(P \times t)$ matrix and $\mathbf{y} = \left[y_T^{(p)}\right]_{p=1}^P$ a $(P \times 1)$ target vector. The difference with the second approach is that now the features for each project are the amount of pledged money at different time step and not the same (shared) input values ($1,...,T$).

Our goal now is to predict, for a new project $p$, the final amount of pledged money $f_*^{(p)} = y_T^{(p)} = f(\mathbf{x}^{(p)}) = f(\mathbf{y}_{1:t}^{(p)})$ for the money received up to time $t$ $X_*^{(p)} = \mathbf{y}_{1:t}^{(p)}$ after observing the total pledged money for all projects $\mathbf{y} = \left[y_T^{(p)}\right]_{p=1}^P$ and the money they received up to time $t$, $X = \left[ \mathbf{y}_{1:t}^{(p)} \right]_{p=1}^P$. In the GP framework, we can compute this prediction using

$$f_* \mid X, \mathbf{y}, X_* \sim \mathcal{N}\left(\overline{f}_*, \text{ cov}(f_*)  \right) \\
\overline{f}_* = K(X_*, X) \left[ K(X, X) + \sigma_n^2I \right]^{-1}\mathbf{y} \\
\text{cov}(f_*) = K(X_*, X_*) - K(X_*, X)\left[ K(X, X) + \sigma_n^2I \right]^{-1}K(X, X_*).
$$ 

As seen in [this plot](http://localhost:8888/notebooks/notebooks/sidekick-1-eda.ipynb#Output-as-input---Global), it seems impossible to do a regression considering all the projects together. However, the two modes corresponding to the successful and failed projects are clearly distinguisable. Therefore we try to perform the regression on each of the two classes separately, using an EM algorithm **[EM DETAILS GO HERE]**

#### Results
Mixture of (GP) models can probably be used.

In [1]:
from __future__ import print_function
%matplotlib inline
import os
import sys
sys.path.insert(0, os.path.abspath('../utils/')) # Add sibling to Python path
sys.path.insert(0, os.path.abspath('../src/')) # Add sibling to Python path
sys.stdout.flush() # Print output on the fly in Notebook
import matplotlib
matplotlib.rcParams['figure.figsize'] = (12,8)
matplotlib.rcParams['font.size'] = 16
matplotlib.rcParams['legend.fontsize'] = 16
from IPython.display import display
import numpy as np
import scipy as sp
import GPy
import climin
import pickle as cp
import matplotlib.pyplot as plt
from math import floor
from dataset import Sidekick
from model.utils.progressbar import ProgressBar

DATA_DIR = "../data/sidekick"

def predict_total_pledged(project, t, samples, m_s_test, m_f_test, pi, X_s, y_s, X_f, y_f):
    """Predict the total pledged money for a given project using a mixture of two GPs:
    
        p(Y_T | y_{1:t}, theta) = pi * p(y_T | y{1:t}, theta_s) + (1-pi) * p(y_T | y{1:t}, theta_f)
        
        where pi is the proportion of successful projects. Returns also the classification.
    """
    money = np.expand_dims(project.money[samples], axis=0)
    X_observed = np.ndarray(shape=(1, t), buffer=money, dtype=float)
    
    l_s = m_s_test.rbf.lengthscale
    sigma_f_s = m_s_test.rbf.variance
    sigma_n_s = m_s_test.Gaussian_noise.variance
    mean_s, var_s, likelihood_s = gaussian_process_regression(X_s, y_s, X_observed, k, 
                                                              l=l_s, 
                                                              sigma_n=sigma_n_s, 
                                                              sigma_f=sigma_f_s)
    l_f = m_f_test.rbf.lengthscale
    sigma_f_f = m_f_test.rbf.variance
    sigma_n_f = m_f_test.Gaussian_noise.variance
    mean_f, var_f, likelihood_f = gaussian_process_regression(X_f, y_f, X_observed, k, 
                                                              l=l_f, 
                                                              sigma_n=sigma_n_f, 
                                                              sigma_f=sigma_f_f)
    y_pred = (mean_s + mean_f) / 2
    likelihood_s = gp_log_likelihood(X_observed, y_pred, l_s, sigma_f_s, sigma_n_s)
    likelihood_f = gp_log_likelihood(X_observed, y_pred, l_f, sigma_f_f, sigma_n_f)
    
    print("Successful:", mean_s, var_s, likelihood_s)
    print("Failed:", mean_f, var_f, likelihood_f)
    print("Predicted: %s (%s)" % (y_pred, "successful" if likelihood_s > likelihood_f else "failed"))
    
    #print(likelihood_s)
    #print(likelihood_f)
    
    return likelihood_s > likelihood_f, y_pred

def subsample(t, granularity):
    if granularity > 1.0 or granularity <= 0:
        raise ValueError("granularity must be in ]0, 1]")
    t0 = 1
    n_samples = int(np.ceil(granularity * t))
    if n_samples == 1:
        return [t]
    else:
        return np.linspace(t0, t, n_samples, dtype=int)


def k(xp, xq , l, sigma_f):
    """Covariance functions with squared exponential of length-scale l and signal noise sigma_f."""
    return sigma_f * np.exp(-0.5 * np.linalg.norm(xp - xq) / float(l**2))
    #return sigma_f * xp * xq


def K(x1, x2, l=1.0, sigma_f=1.0):
    """Compute the covariance matrix from the observations x."""
    cov_matrix = np.zeros((len(x1), len(x2)))
    for i, p in enumerate(x1):
        for j, q in enumerate(x2):
            cov_matrix[i, j] = k(p, q, l, sigma_f)
    return cov_matrix

def gp_log_likelihood(x, y, l=1.0, sigma_n=0.0, sigma_f=1.0):
    n = len(x)
    L = np.linalg.cholesky(K(x, x, l, sigma_f) + sigma_n * np.eye(n))
    a = sp.linalg.solve_triangular(L.T, sp.linalg.solve_triangular(L, y, lower=True))
    lml = -0.5 * np.sum(y * a) - np.sum(np.log(np.diag(L))) - n * 0.918938533205
    return lml


def gaussian_process_regression(X, y, x_test, k, l=1.0, sigma_n=0.0, sigma_f=1.0):
    """
    Computes a regression using Gaussian Process using observations x and y = f(x) and a covariance function k.
    
    :param X        Inputs (NxD)
    :param y        Values at indices x (= f(x))
    :param x_test   Indices to get predicted values
    :param k        Covariance function
    :param sigma_n  Observations noise
    :return:        Mean m, variance var and log marginal likelihood lml
    """
    n = len(y)
    n_test = len(x_test)
    # Cholesky decompostion
    L = np.linalg.cholesky(K(X, X, l, sigma_f) + sigma_n * np.eye(n))
    a = sp.linalg.solve_triangular(L.T, sp.linalg.solve_triangular(L, y, lower=True))
    k_star = K(X, x_test, l, sigma_f)
    # Predictive mean
    m = np.dot(k_star.T, a)
    # Predictive variance
    v = sp.linalg.solve_triangular(L, k_star, lower=True)
    var = K(x_test, x_test, l, sigma_f) - np.dot(np.transpose(v), v)
    # Log maginal likelihood (last term is log2π / 2)
    #lml = -0.5 * np.sum(y * a) - np.sum(np.log(np.diag(L))) - n * 0.918938533205
    lml = 0
    return m, var + sigma_n * np.eye(n_test), lml

warning in stationary: failed to import cython module: falling back to numpy


## Load data and separate successful from failed projects

In [2]:
sk = Sidekick(seed=2)
sk.load(light=False)
# projects_train, projects_test = sk.split()
# successful = sk.successful()
# failed = sk.failed()

Loading data set...
Data loaded.


### Train GP on all the projects

In [ ]:
projects_train, projects_test = sk.split(shuffle=False)

t = 25
granularity = 0.001
samples = subsample(t, granularity)
n_samples = len(samples)

T = 999
ARD = True
outlier_threshold = 10000

projects_train_filtered = [p for p in projects_train if np.all((p.money[T] * p.goal - outlier_threshold) <= 0) and np.all((p.money[samples]* p.goal - outlier_threshold) <= 0)]
projects_test_filtered = [p for p in projects_test if np.all((p.money[T] * p.goal - outlier_threshold) <= 0) and np.all((p.money[samples]* p.goal - outlier_threshold) <= 0)]

X_train = np.ndarray(shape=(len(projects_train_filtered), n_samples), buffer=np.array([p.money[samples] * p.goal for p in projects_train_filtered]), dtype=float)
y_train = np.expand_dims(np.array([p.money[T] * p.goal for p in projects_train_filtered]), axis=1)
X_test = np.ndarray(shape=(len(projects_test_filtered), n_samples), buffer=np.array([p.money[samples] * p.goal for p in projects_test_filtered]), dtype=float)
y_test = np.expand_dims(np.array([p.money[T] * p.goal for p in projects_test_filtered]), axis=1)

#c = 0.1
#X_train = np.log(X_train + c)
#X_test = np.log(X_test + c) 
       
print(X_train.shape)

iterations = 1000
scaling = 4000
num_inducing_points = 100
Z = np.random.rand(num_inducing_points, 1) * scaling # Inducing points
batchsize = 100

kernel = GPy.kern.RBF(input_dim=n_samples, ARD=ARD)
m = GPy.core.SVGP(X_train, y_train, Z, kernel, GPy.likelihoods.Gaussian(), batchsize=batchsize)
#m.kern.white.variance = 1e-5 # Random noise to stabilize the inversion ("fixed jitter")
#m.kern.white.fix()
# + GPy.kern.White(1)

opt = climin.Adadelta(m.optimizer_array, m.stochastic_grad, step_rate=0.8, momentum=0.9)
from ipywidgets import Text
from IPython.display import display

text = Text(description="Likelihood:", width=400)
display(text)

import sys
def callback(i):
    text.value = str(m.log_likelihood()) + "(i=%s)" % i['n_iter']
    #Stop after M iterations
    if i['n_iter'] > iterations:
        print("Hitting maximum iteration.")
        return True
    return False
info = opt.minimize_until(callback)

#m = GPy.models.GPRegression(X_train, y_train, kernel)
#m.optimize()

display(m)

ax = plt.gca()
ax.plot(X_train, y_train, 'kx', alpha=0.2, label="Data")
ax.set_xlabel('Money at time %s' % t)
ax.set_ylabel('Goal')
ax.set_title('SVI Y prediction with data')
m.plot(plot_limits=(X_train.min(),X_train.max()), ax=ax)
ax.set_xlim((X_train.min(),X_train.max()))
plt.legend(['Training Data', 'Mean'], numpoints=1, loc='best')
plt.show()

if n_samples == 2: 
    plot = m.plot()
    plt.xlabel("Money pledged at time %s" % t)
    plt.ylabel("Total pldeged money")
    plt.title("Total pledged money for each project given the amount at different time")
    #plt.xlim([0, 1.2])
    #plt.ylim([0, outlier_threshold])
    plt.legend(loc='best')
    plt.savefig("gp_output_input.pdf")

### Test prediction for one project

In [ ]:
test = 1
project_test = projects_test_filtered[test]
X_observed = np.expand_dims(X_test[test], axis=0)
y_pred, var = m.predict(X_observed)

#print "Predicted as %0.4f (%0.4f)" % (yT_mean, yT_std)
print("Predicted: %.2f (%.2f)" % (y_pred[0,0], var[0, 0]))
print("Actual: %0.2f" % (y_test[test]))
project_test.plot()

### Evaluate on test set

In [4]:
se_successful = []
se_failed = []
se_total = []
accuracy = 0
for i, x_test in enumerate(X_test):
    p = projects_test_filtered[i]
    goal = float(p.goal)
    x_test = np.expand_dims(x_test, axis=0)
    y_pred, y_var = m.predict(x_test)
    y_pred = y_pred[0, 0]
    y_var = y_var[0, 0]
    y_actual = y_test[i]
    se = (y_pred - y_actual)**2
    se_total.append(se)
    if y_test[i] >= 1.0:  # Project is successful
        se_successful.append(se)
    else:  # project is failed
        se_failed.append(se)
    if (y_pred / goal >= 1 and y_actual / goal >= 1) or (y_pred / goal < 1 and y_actual / goal < 1):
        accuracy += 1
    #else:
    #    print("Error:", i)

rmse_successful = np.sqrt(np.mean(se_successful))
rmse_failed = np.sqrt(np.mean(se_failed))
rmse_total = np.sqrt(np.mean(se_total))
accuracy /= float(len(y_test))

print("Accuracy: ", accuracy)
print("RMSE: ", rmse_total)
print("RMSE successful: ", rmse_successful)
print("RMSE failed: ", rmse_failed)


Accuracy:  0.600523168909
RMSE:  3017.91106705
RMSE successful:  3144.42347884
RMSE failed:  0.0


### Parameters

In [ ]:
N = 1000
seed = 2

t = 25
granularity = 0.001
samples = subsample(t, granularity)
n_samples = len(samples)
T = 999
ARD = False
outliers_threshold = 10000

N_train = int(floor(0.8*N))
N_test = N - N_train

N_projects = sk.choose_n_projects(n=N)
projects_train = N_projects[:N_train]
projects_test = N_projects[N_train:]
successful = [p for p in projects_train if p.successful and p.money[T] * p.goal < outliers_threshold]
failed = [p for p in projects_train if not p.successful and p.money[T] * p.goal < outliers_threshold]
pi = len(successful) / float(N_train)

print("Successful: %s (train)" % len(successful))
print("Failed: %s (train)" % len(failed))
print("p(successful) = %0.4f (prior)" % pi)

## Train GP-RBF on successful projects

In [ ]:
X_train_s = np.ndarray(shape=(len(successful), n_samples), buffer=np.array([p.money[samples] * p.goal for p in successful]), dtype=float) 
Y_train_s = np.expand_dims(np.array([p.money[T] * p.goal for p in successful]), axis=1)
print(X_train_s.shape)
print(Y_train_s.shape)

kernel = GPy.kern.RBF(input_dim=n_samples, ARD=ARD)
m_s = GPy.models.GPRegression(X_train_s, Y_train_s, kernel)
m_s.optimize()
#m_s.rbf.lengthscale = 0.01
display(m_s)
m_s.plot()

In [ ]:
def view_dimension(d, total):
    inputs = range(total)
    inputs.pop(d)
    fixed_inputs = [(i, 0) for i in inputs]
    return fixed_inputs

print(m_s.rbf.variance)
print(m_s.rbf.lengthscale)
f = view_dimension(9, t)
fig = m_s.plot(fixed_inputs=f)

## Train GP-RBF on failed projects

In [ ]:
X_train_f = np.ndarray(shape=(len(failed), n_samples), buffer=np.array([p.money[samples] * p.goal for p in failed]), dtype=float) 
Y_train_f = np.expand_dims(np.array([p.money[T] * p.goal for p in failed]), axis=1)
print(X_train_f.shape)
print(Y_train_f.shape)

kernel = GPy.kern.RBF(input_dim=n_samples, ARD=ARD)
m_f = GPy.models.GPRegression(X_train_f, Y_train_f, kernel)
m_f.optimize()
display(m_f)
m_f.plot()

In [ ]:
print m_f.rbf.variance
print m_f.rbf.lengthscale
f = view_dimension(2, t)
fig = m_f.plot(fixed_inputs=f)

## Test experiment

In [ ]:
m_s_test = m_s.copy()
m_f_test = m_f.copy()
project_test = projects_test[0]

success, y_pred = predict_total_pledged(project_test, n_samples, samples, m_s_test, m_f_test, pi, X_train_s, Y_train_s, X_train_f, Y_train_f)
#print "Predicted as %0.4f (%0.4f)" % (yT_mean, yT_std)
correct = (project_test.successful and success) or (not project_test.successful and not success)
print("Classified as %s, %s" % ("successful" if success else "failed", "CORRECT!" if correct else "wrong..."))
print("Actual total pledged money: %0.4f" % (project_test.money[T] * project_test.goal))
project_test.plot()

## Run experiment

In [ ]:
m_s_test = m_s.copy()
m_f_test = m_f.copy()
se_successful = 0
std_successful = 0
se_failed = 0
std_failed = 0
loss01 = 0
count_classifier_successful = 0
total_considered = 0
for project in projects_test:
    if project.money[T] < outliers_threshold:
        total_considered += 1
        success, y_pred = predict_total_pledged(project, t, samples, m_s_test, m_f_test, pi, X_train_s, Y_train_s, X_train_f, Y_train_f)
        if project.successful: 
                #se_successful += (project.money[T] - y_T_mean)**2
                #std_successful += yT_std
                if not success:  # Classification error
                    loss01 += 1
        else:  # project is failed
            #se_failed += (project.money[T] - y_T_mean)**2
            #std_failed += yT_std
            if success:  # Classification error
                loss01 += 1
        if success:
            count_classifier_successful += 1
        
#print "RMSE = %0.4f (±%0.4f)" % (np.sqrt(np.mean(se_successful)), std_successful / float(N_test))
print("01-Loss = %0.4f" % (loss01 / float(N_test)))
print("Classifed as successful: %s out of %s projects considered" % (count_classifier_successful, total_considered))